---
jupyter:
  kernelspec:
    name: ir
    display_name: R
    language: R
---

In [ ]:
# Install necessary packages
if (!require(tseries)) {  # Check if 'tseries' package is installed
  install.packages("tseries")  # Install the 'tseries' package if not already installed
  library(tseries)  # Load the 'tseries' package
}
if (!require(googledrive)) {  # Check if 'googledrive' package is installed
  install.packages("googledrive")  # Install the 'googledrive' package if not already installed
  library(googledrive)  # Load the 'googledrive' package
}

# Authorize access to Google Drive
drive_auth()

In [ ]:
# Search for the file on Google Drive
file <- drive_get("Data_Total.csv")

# Download the file to the temporary system
drive_download(file, path = "Data_Total.csv", overwrite = TRUE)

# Read the data from the CSV file and assign it to stations_data
stations_data <- read.csv("Data_Total.csv")

# Convert the date column to Date format
stations_data$DATE <- as.Date(stations_data$DATE, format = "%Y-%m-%d")

# Filter the data starting from the date "1955-01-01"
stations_data <- subset(stations_data, DATE >= as.Date("1955-01-01"))

# Check the structure of the data
str(stations_data)

# Summarize the data
summary(stations_data)

# Display a preview of the data
head(stations_data)

# Verify the data filtering
cat("Number of rows after filtering:", nrow(stations_data), "\n")

# Summarize the data after filtering
summary(stations_data)

In [ ]:
# Rename rows in the NAME column
stations_data$NAME <- gsub("LUGANO, SZ", "Lugano", stations_data$NAME)
stations_data$NAME <- gsub("SAENTIS, SZ", "Saentis", stations_data$NAME)
stations_data$NAME <- gsub("GENEVE COINTRIN, SZ", "Genève", stations_data$NAME)

# Keep only the three specified stations: Lugano, Saentis, Genève
stations_data <- subset(stations_data, NAME %in% c("Lugano", "Saentis", "Genève"))

# Check the data after filtering
cat("Number of rows after filtering:", nrow(stations_data), "\n")
unique(stations_data$NAME)

# Export the cleaned data to a CSV file
write.csv(stations_data, "Cleaned_Stations_Data.csv", row.names = FALSE)

cat("Cleaned file exported as 'Cleaned_Stations_Data.csv'.\n")

In [ ]:
# Check the structure of the data
str(stations_data)

# Summarize the data
summary(stations_data)

# Display a preview of the data
head(stations_data)

In [ ]:
# Load ggplot2 if not already loaded
if (!require(ggplot2)) {
  install.packages("ggplot2")
  library(ggplot2)
}

# Create a plot for each station and display them separately
stations <- unique(stations_data$NAME)

for (station in stations) {
  plot <- ggplot(subset(stations_data, NAME == station), aes(x = DATE, y = TMAX)) +
    geom_line(color = "blue") +
    labs(
      title = paste("Maximum Temperature Over Time -", station),
      x = "Date",
      y = "Maximum Temperature (°C)"
    ) +
    theme_minimal() +
    theme(
      plot.title = element_text(hjust = 0.5)
    )

  print(plot)  # Display each plot
}

In [ ]:
# Create a histogram for each station
for (station in stations) {
  plot <- ggplot(subset(stations_data, NAME == station), aes(x = TMAX)) +
    geom_histogram(binwidth = 1, fill = "blue", color = "black", alpha = 0.7) +
    labs(
      title = paste("Histogram of Maximum Temperatures -", station),
      x = "Maximum Temperature (°C)",
      y = "Frequency"
    ) +
    theme_minimal() +
    theme(
      plot.title = element_text(hjust = 0.5)
    )

  print(plot)  # Display each histogram
}

In [ ]:
# Check for missing values
na_counts <- colSums(is.na(stations_data))
cat("Number of missing values per column before removal:\n")
print(na_counts)

# Remove only rows where TMAX is NA
stations_data <- stations_data[!is.na(stations_data$TMAX), ]

# Check for missing values again
na_counts_after <- colSums(is.na(stations_data))
cat("\nNumber of missing values per column after removal:\n")
print(na_counts_after)

# Split the data by station
stations_split <- split(stations_data, stations_data$NAME)

# Count the number of values per station
values_per_station <- sapply(stations_split, nrow)
cat("\nNumber of remaining values per station after removing NAs in TMAX:\n")
print(values_per_station)

# Display the result for each station
for (station in names(values_per_station)) {
  cat("\nStation:", station, " | Number of rows:", values_per_station[station], "\n")
}

In [ ]:
# We will create a subset for each station.

# Check the unique station names to confirm
unique(stations_data$NAME)

# Split the data by station
lugano_data <- subset(stations_data, NAME == "Lugano")
genève_data <- subset(stations_data, NAME == "Genève")
saentis_data <- subset(stations_data, NAME == "Saentis")

In [ ]:
# Check the dimensions of the subsets
cat("Number of rows for Lugano:", nrow(lugano_data), "\n")
cat("Number of rows for Genève:", nrow(genève_data), "\n")
cat("Number of rows for Säntis:", nrow(saentis_data), "\n")

In [ ]:
# Function to explore a station
explore_station <- function(data, station_name) {
  cat("\n--- Exploration for station:", station_name, "---\n")

  # Preview of the data
  cat("\nData preview:\n")
  print(head(data))

  # Descriptive statistics
  cat("\nDescriptive statistics:\n")
  print(summary(data))

  # Checking for missing values
  cat("\nNumber of missing values per column:\n")
  print(colSums(is.na(data)))

  # Histogram of maximum temperatures
  cat("\nHistogram of maximum temperatures for:", station_name, "\n")
  print(
    ggplot(data, aes(x = TMAX)) +
      geom_histogram(binwidth = 1, fill = "red", color = "black") +
      labs(title = paste("Distribution of Maximum Temperatures -", station_name),
           x = "Maximum Temperature (°C)", y = "Frequency") +
      theme_minimal()
  )

  # Time series of maximum temperatures
  cat("\nTime series of maximum temperatures for:", station_name, "\n")
  print(
    ggplot(data, aes(x = DATE, y = TMAX)) +
      geom_line(color = "red") +
      labs(title = paste("Maximum Temperatures Over Time -", station_name),
           x = "Date", y = "Maximum Temperature (°C)") +
      theme_minimal()
  )
}

# Exploration for Lugano
explore_station(lugano_data, "Lugano")

# Exploration for Genève
explore_station(genève_data, "Genève")

# Exploration for Saentis
explore_station(saentis_data, "Saentis")

Now we'll only focus on the summer period.


In [ ]:
# Filter summer data for Lugano
lugano_summer <- subset(lugano_data, format(DATE, "%m") %in% c("06", "07", "08", "09"))

# Filter summer data for Genève
genève_summer <- subset(genève_data, format(DATE, "%m") %in% c("06", "07", "08", "09"))

# Filter summer data for Säntis
saentis_summer <- subset(saentis_data, format(DATE, "%m") %in% c("06", "07", "08", "09"))

# Check the dimensions of the summer subsets
cat("Number of summer rows for Lugano:", nrow(lugano_summer), "\n")
cat("Number of summer rows for Genève:", nrow(genève_summer), "\n")
cat("Number of summer rows for Säntis:", nrow(saentis_summer), "\n")

In [ ]:
# Plot the evolution of summer maximum temperatures for Lugano
plot(lugano_summer$DATE, lugano_summer$TMAX, type = "l", col = "red",
     main = "Evolution of Summer Maximum Temperatures - Lugano",
     xlab = "Date", ylab = "Maximum Temperature (°C)")

# Plot the evolution of summer maximum temperatures for Genève
plot(genève_summer$DATE, genève_summer$TMAX, type = "l", col = "blue",
     main = "Evolution of Summer Maximum Temperatures - Genève",
     xlab = "Date", ylab = "Maximum Temperature (°C)")

# Plot the evolution of summer maximum temperatures for Säntis
plot(saentis_summer$DATE, saentis_summer$TMAX, type = "l", col = "green",
     main = "Evolution of Summer Maximum Temperatures - Säntis",
     xlab = "Date", ylab = "Maximum Temperature (°C)")

## Now perform cluster analysis, etc., as in TP2


In [ ]:
install.packages("extRemes", dependencies = TRUE)

In [ ]:
# Install the package if it is not already installed
if (!require(extRemes)) {
  install.packages("extRemes", dependencies = TRUE)
}

# Load the package
library(extRemes)

In [ ]:
# Define a threshold for extreme temperatures
# We can define a threshold based on the 95th percentile of summer maximum temperatures for each station.

# Calculate thresholds for each station
threshold_lugano <- quantile(lugano_summer$TMAX, 0.95, na.rm = TRUE)
threshold_genève <- quantile(genève_summer$TMAX, 0.95, na.rm = TRUE)
threshold_saentis <- quantile(saentis_summer$TMAX, 0.95, na.rm = TRUE)

cat("Threshold for Lugano:", threshold_lugano, "\n")
cat("Threshold for Genève:", threshold_genève, "\n")
cat("Threshold for Säntis:", threshold_saentis, "\n")

In [ ]:
# Load the extRemes package if necessary
if (!require(extRemes)) {
  install.packages("extRemes", dependencies = TRUE)
  library(extRemes)
}

# Validation with the MRL Plot for Lugano
cat("Mean Residual Life Plot for Lugano\n")
mrlplot(lugano_summer$TMAX, main = "Mean Residual Life Plot - Lugano")

# Validation with the MRL Plot for Genève
cat("Mean Residual Life Plot for Genève\n")
mrlplot(genève_summer$TMAX, main = "Mean Residual Life Plot - Genève")

# Validation with the MRL Plot for Säntis
cat("Mean Residual Life Plot for Säntis\n")
mrlplot(saentis_summer$TMAX, main = "Mean Residual Life Plot - Säntis")

The Mean Residual Life (MRL) plot is used to validate the choice of thresholds for defining extreme events. It shows the mean excess, or the average amount by which temperatures exceed a threshold, as the threshold increases. For Geneva for example, in the plot, the curve decreases steeply at lower thresholds, indicating the inclusion of non-extreme values. Around 31°C, the curve stabilizes, suggesting this is a suitable threshold for defining extremes. The stability and narrow confidence bands confirm the reliability of this threshold. At higher thresholds, wider confidence intervals indicate less reliable estimates, further supporting the choice of 31°C as appropriate for the analysis.


In [ ]:
# Identify extreme days for Lugano
lugano_extreme_days <- lugano_summer[lugano_summer$TMAX > threshold_lugano, ]
cat("Number of extreme days for Lugano:", nrow(lugano_extreme_days), "\n")

# Identify extreme days for Genève
geneve_extreme_days <- genève_summer[genève_summer$TMAX > threshold_genève, ]
cat("Number of extreme days for Genève:", nrow(geneve_extreme_days), "\n")

# Identify extreme days for Säntis
saentis_extreme_days <- saentis_summer[saentis_summer$TMAX > threshold_saentis, ]
cat("Number of extreme days for Säntis:", nrow(saentis_extreme_days), "\n")

In [ ]:
# Calculation of the extremal index for each station
extremal_index_lugano <- extremalindex(lugano_summer$TMAX, threshold = threshold_lugano)
extremal_index_geneve <- extremalindex(genève_summer$TMAX, threshold = threshold_genève)
extremal_index_saentis <- extremalindex(saentis_summer$TMAX, threshold = threshold_saentis)

cat("Extremal index for Lugano:", extremal_index_lugano, "\n")
cat("Extremal index for Genève:", extremal_index_geneve, "\n")
cat("Extremal index for Säntis:", extremal_index_saentis, "\n")

These results represent the extremal index calculations for each station, which measure the tendency of extreme events to occur in clusters. Here's what the numbers mean:

Extremal Index (First Number):

For Lugano: 0.2007
For Genève: 0.2699
For Säntis: 0.3573
The extremal index ranges between 0 and 1:

A value close to 0 indicates that extreme events are strongly clustered.
A value closer to 1 suggests that extreme events are more isolated and independent.
In this case, Lugano shows the most clustering of extremes (lowest index), while Säntis has more independent extreme events (highest index).

Number of Clusters (Second Number):

For Lugano: 80 clusters
For Genève: 112 clusters
For Säntis: 148 clusters
This indicates the number of distinct clusters of extreme events detected for each station. More clusters suggest a higher frequency of extremes occurring over the studied period.

Run Length (Third Number):

For Lugano: 17
For Genève: 8
For Säntis: 8
This reflects the "run parameter," which defines the minimum separation between events for them to be considered independent clusters. Higher values mean the analysis allows for longer gaps between clustered events.

Interpretation
Lugano's low extremal index (0.2007), combined with a relatively low number of clusters (80), indicates strongly clustered extreme events.
Genève has a slightly higher extremal index (0.2699) and more clusters (112), suggesting less clustering than Lugano.
Säntis shows the highest extremal index (0.3573) and the most clusters (148), indicating that extremes here are more independent and less likely to occur in tightly grouped clusters.
These differences likely reflect local climatic and geographic conditions influencing how extreme temperatures occur.


In [ ]:
# Load necessary library
if (!require(extRemes)) install.packages("extRemes", dependencies = TRUE)
library(extRemes)

# Decluster the data using the chosen threshold for each station
lugano_declustered <- decluster(lugano_summer$TMAX, threshold = threshold_lugano, run.length = 1)
genève_declustered <- decluster(genève_summer$TMAX, threshold = threshold_genève, run.length = 1)
saentis_declustered <- decluster(saentis_summer$TMAX, threshold = threshold_saentis, run.length = 1)

# Add the declustered data to the corresponding datasets
lugano_summer$Declustered <- ifelse(lugano_summer$TMAX >= threshold_lugano, lugano_declustered, NA)
genève_summer$Declustered <- ifelse(genève_summer$TMAX >= threshold_genève, genève_declustered, NA)
saentis_summer$Declustered <- ifelse(saentis_summer$TMAX >= threshold_saentis, saentis_declustered, NA)

In [ ]:
# Visualiser les données déclusterisées

# Visualisation pour Lugano
ggplot() +
  geom_line(data = lugano_summer, aes(x = DATE, y = TMAX), color = "blue", alpha = 0.5) +
  geom_point(data = subset(lugano_summer, !is.na(Declustered)), aes(x = DATE, y = Declustered), color = "red") +
  labs(title = "Declustered Temperatures - Lugano", x = "Date", y = "Max Temperature (°C)") +
  theme_minimal()

# Visualisation pour Genève
ggplot() +
  geom_line(data = genève_summer, aes(x = DATE, y = TMAX), color = "blue", alpha = 0.5) +
  geom_point(data = subset(genève_summer, !is.na(Declustered)), aes(x = DATE, y = Declustered), color = "red") +
  labs(title = "Declustered Temperatures - Genève", x = "Date", y = "Max Temperature (°C)") +
  theme_minimal()

# Visualisation pour Säntis
ggplot() +
  geom_line(data = saentis_summer, aes(x = DATE, y = TMAX), color = "blue", alpha = 0.5) +
  geom_point(data = subset(saentis_summer, !is.na(Declustered)), aes(x = DATE, y = Declustered), color = "red") +
  labs(title = "Declustered Temperatures - Säntis", x = "Date", y = "Max Temperature (°C)") +
  theme_minimal()

Explanation
Objective:
Declustering helps remove dependencies between extreme events that occur in close succession, ensuring that only independent extreme events are considered for further analysis. This is important for correctly modeling extreme behavior and deriving accurate probabilities for rare events.

Steps:

Threshold Selection:
Using the thresholds previously validated with the Mean Residual Life (MRL) plot, we identify extreme events for each station.

Declustering:
Using the decluster function from the extRemes package, we filter the extreme values such that only the first event in each cluster is retained. The run.length parameter determines the minimum separation in days between clusters. For example, a run.length of 1 means any extreme values occurring consecutively are considered part of the same cluster.

Visualization:

The gray line represents the original maximum temperatures over time.
The red, blue, or green points represent the declustered extreme values. These are independent extremes retained after declustering.
The resulting plot allows us to visually verify how declustering isolates the independent extremes while removing consecutive dependencies.

# fit a Generalized Pareto Distribution (GPD) to the declustered extreme temperatures and estimate probabilities and return levels:


In [ ]:
# Ensure the extRemes package is installed
if (!require(extRemes)) install.packages("extRemes", dependencies = TRUE)
library(extRemes)

In [ ]:
# Define a function to fit the GPD model and calculate return levels
fit_gpd <- function(data, declustered_column, threshold, return_period) {
  # Extract declustered extreme values
  extreme_values <- data[[declustered_column]][!is.na(data[[declustered_column]])]

  # Fit the Generalized Pareto Distribution (GPD)
  gpd_fit <- fevd(extreme_values, type = "GP", threshold = threshold)

  # Calculate the return level for the specified return period
  return_level <- return.level(gpd_fit, return.period = return_period)

  # Return the GPD fit and return level
  list(fit = gpd_fit, return_level = return_level)
}

# Fit GPD for Lugano
gpd_lugano <- fit_gpd(lugano_summer, "Declustered", threshold_lugano, return_period = 10)

# Fit GPD for Genève
gpd_genève <- fit_gpd(genève_summer, "Declustered", threshold_genève, return_period = 10)

# Fit GPD for Säntis
gpd_saentis <- fit_gpd(saentis_summer, "Declustered", threshold_saentis, return_period = 10)

In [ ]:
# Print summaries and return levels
cat("Lugano - 10-Year Return Level:", gpd_lugano$return_level, "\n")
cat("Genève - 10-Year Return Level:", gpd_genève$return_level, "\n")
cat("Säntis - 10-Year Return Level:", gpd_saentis$return_level, "\n")

# Plot the GPD model fit for each station
par(mfrow = c(1, 3))  # Arrange plots in a single row
plot(gpd_lugano$fit, main = "GPD Fit - Lugano")
plot(gpd_genève$fit, main = "GPD Fit - Genève")
plot(gpd_saentis$fit, main = "GPD Fit - Säntis")

To calculate the probability that an extreme day is followed by another extreme day for each station, we use the extremal index, which measures the degree of clustering of extremes. The extremal index can be interpreted as the inverse of the average cluster size. Here's the code to compute this for each station:


In [ ]:
# Compute the probability of consecutive extreme days for each station
compute_extreme_following_prob <- function(extremal_index) {
  return(1 - extremal_index) # Probability of consecutive extremes
}

# For Lugano
extremal_index_lugano <- extremalindex(lugano_summer$TMAX, threshold = threshold_lugano)
prob_consecutive_extremes_lugano <- compute_extreme_following_prob(extremal_index_lugano[1])
cat("Probability of consecutive extremes for Lugano:", prob_consecutive_extremes_lugano, "\n")

# For Genève
extremal_index_genève <- extremalindex(genève_summer$TMAX, threshold = threshold_genève)
prob_consecutive_extremes_genève <- compute_extreme_following_prob(extremal_index_genève[1])
cat("Probability of consecutive extremes for Genève:", prob_consecutive_extremes_genève, "\n")

# For Saentis
extremal_index_saentis <- extremalindex(saentis_summer$TMAX, threshold = threshold_saentis)
prob_consecutive_extremes_saentis <- compute_extreme_following_prob(extremal_index_saentis[1])
cat("Probability of consecutive extremes for Saentis:", prob_consecutive_extremes_saentis, "\n")